In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import json

# Basic web scraping

In [ ]:
base_url = 'https://www.jamieoliver.com/search/?s='
search = 'mexican'
url = base_url+search

In [ ]:
html_page = urlopen(url)
parsed_page = BeautifulSoup(html_page, 'html.parser')

In [ ]:
recipe_url_list = []

for link in parsed_page.find_all('a'):
    recipe_url = str(link.get('href'))
    if "jamieoliver.com/recipes" in recipe_url and 'category' not in recipe_url:
        recipe_url_list.append(recipe_url)

In [ ]:
recipe_url = recipe_url_list[0]
recipe_url

In [ ]:
recipe_name_list = []
recipes={}

for recipe_url in recipe_url_list:
    
    # scrape the webpage
    try:
        hmtl_recipe = urlopen(recipe_url, timeout=5)
    except HTTPError or URLError:
        break
        
    parsed_recipe = BeautifulSoup(hmtl_recipe, 'html.parser')
    simple_details = parsed_recipe.find(attrs={'class':'single-recipe-details-mobile-top'})
    try:
        recipe_name = simple_details.find('h3').text
        recipe_name_extra = simple_details.find('p').text
    except AttributeError:
        recipe_name_extra = ""
    
    name = recipe_name+", " + recipe_name_extra
    recipe_name_list.append(name)
    recipes[name] = [parsed_recipe]
    
    time.sleep(1)

In [ ]:
for key in recipes:
    #parsed_recipe = recipes[key]
    try:
        instructions_text = recipes[key].find(attrs={'class':'recipeSteps'}).text
        ingredient_list_text = recipes[key].find(attrs={'class':'ingred-list'}).text
    except AttributeError:
        continue

In [ ]:
instructions_text

In [ ]:
recipes[key]

# Data Wrangling

In [ ]:
def getingredients(ingredient_list_text):


    ingredient_dict={}
    
    for ingredient_line in ingredient_list_text.split(sep="\n\n"):
        # if its empty then dont do anything
        if len(ingredient_line)==0:
            continue
        
        # split the ingredient into words, the original ingredient_line has lots of white space
        # split and then reset it to build it back
        ingredient_line_split = ingredient_line.split()
        ingredient_line=" "
        
        
        # for each word in the ingredient line check if it's a number or if it's in the ingredient corpus
        for word in ingredient_line_split:
            ingredient_line+=word+" "
            ingredient=""
            
            try:
                quantity = int(word)
            except ValueError:
                if word in ingredient_corpus:
                    ingredient += word + " "
                
        ingredient_dict[ingredient_line]=[ingredient,quantity]
        
    return ingredient_dict

In [ ]:
ingredient_dict = getingredients(ingredient_list_text)
ingredient_dict

# Ingredient Corpus

In [ ]:
def get_json_ingreds(file):

    ingredient_corpus = []
    
    with open(file) as json_data:
        ingred_dict = json.load(json_data)
    for item in ingred_dict:
        ingredient_corpus.extend(item["ingredients"])
        
    ingredient_corpus = list(set(ingredient_corpus))
    
    return ingredient_corpus

In [ ]:
ingredient_corpus = get_json_ingreds('data/ingredients.json')
ingredient_corpus.extend(get_json_ingreds('data/ingredients2.json'))

ingredient_corpus = list(set(ingredient_corpus))

In [ ]:
for word in ingredients_split:
    if word in ingredient_corpus:
        print(word)

In [ ]:
ingredient_line

In [ ]:
for item in ingredient_corpus:
    if "coriander" in item:
        print(item)

In [ ]:
ingredients_split